In [1]:
# Imports

import os
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from chunker import Chunker

In [2]:
# Save the model weights in this directory
weight_dir = "model_wts"

if not os.path.exists(weight_dir):
  os.mkdir(weight_dir)

In [ ]:
# Functions to generate the sample from the CSV file
chunk_size = 512
batch_size = 16
chromatic  = True
noise = 0.5
num_chunks = 10

chunker = Chunker('./bach_CSV', chunk_size, batch_size, chromatic, noise)

In [9]:
# Generate and concatenate chunks
targets = []
samples = []

for _ in range(num_chunks):
    target, sample = chunker.create_chunck()
    targets.append(torch.tensor(target, dtype=torch.float32))
    samples.append(torch.tensor(sample, dtype=torch.float32))

# Concatenate all targets and samples
targets_tensor = torch.cat(targets)
samples_tensor = torch.cat(samples)


ValueError: could not broadcast input array from shape (0,12) into shape (16,12)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ChunkTensorDataset(Dataset):
    def __init__(self, targets_tensor, samples_tensor):
        self.targets_tensor = targets_tensor
        self.samples_tensor = samples_tensor

    def __len__(self):
        return len(self.targets_tensor)

    def __getitem__(self, idx):
        return self.targets_tensor[idx], self.samples_tensor[idx]
    
# Create Dataset and DataLoader
dataset = ChunkTensorDataset(targets_tensor, samples_tensor)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [5]:
# Variables
input_size = 12
num_layers = 2
hidden_size = 64
learning_rate = 0.001
num_epochs = 50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size):
        super(SimpleRNN, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
          batch_size = x.size(0)
          hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.rnn(x, hidden)
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = SimpleRNN(input_size, num_layers, hidden_size).to(device=device)

# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")


# Save the weights
weights_file = os.path.join(dir, 'SimpleRNNweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"Simple RNN weights saved to {dir}")

In [14]:
class GRUModel(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size):
        super(GRUModel, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
            batch_size = x.size(0)
            hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.gru(x, hidden)
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = GRUModel(input_size, num_layers, hidden_size).to(device=device)


# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")


# Save the weights
weights_file = os.path.join(dir, 'GRUweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"GRU Model weights saved to {dir}")

0it [00:00, ?it/s]


AttributeError: 'Chunker' object has no attribute 'create_chunk'

In [ ]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size):
        super(SimpleLSTM, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
          batch_size = x.size(0)
          hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
          c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.lstm(x, (hidden, c0))
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = SimpleLSTM(input_size, num_layers, hidden_size).to(device=device)

# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")

# Save the weights
weights_file = os.path.join(dir, 'LSTMweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"LSTM Model weights saved to {dir}")